In [ ]:
import logging
import os
import queue
import random
import re
from args import get_setup_args
import shutil
import string
#import setup
import torch
import torch.nn.functional as F
import torch.utils.data as data
from collections import Counter
import numpy as np
import ujson as json
#import spacy
import json
import tensorflow as tf

from tqdm import tqdm_notebook as tqdm

In [ ]:
adversarial_data = np.load('./sentence_selection_data/train_sent_sel_adv.npz')
train_q = adversarial_data['train_q']
train_s = adversarial_data['train_s']
train_labels = adversarial_data['train_labels']

In [ ]:
from tensorflow.keras.layers import Dropout, concatenate, LSTM, Dense, concatenate

class QUA_Net(tf.keras.Model):
    def __init__(self):
        super(QUA_Net, self).__init__()
        self.q_lstm = LSTM(128)
        self.s_lstm = LSTM(128)
        self.dropout = Dropout(0.5)
        self.dense = Dense(1,activation='sigmoid')
        
    def call(self, inputs):
        q = inputs[0]
        s = inputs[1]
        q_out = self.q_lstm(q)
        s_out = self.s_lstm(s)
        
        merge = concatenate([q_out, s_out], axis = -1)
        drop = self.dropout(merge)
        out = self.dense(drop)
        return out

model = QUA_Net()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
pos = np.where(train_labels == True)
print(len(pos[0]))
pos_idxs = pos[0]

negs = np.where(train_labels == False)
print(len(negs[0]))

neg_idxs = np.random.choice(negs[0], len(pos[0]))
print(len(neg_idxs), neg_idxs)
print(train_q[0].shape)
train_balanced_q = np.take(train_q, np.append(pos_idxs, neg_idxs), axis=0)
train_balanced_s = np.take(train_s, np.append(pos_idxs, neg_idxs), axis=0)
train_balanced_labels = np.take(train_labels, np.append(pos_idxs, neg_idxs), axis=0)
print(train_balanced_q.shape)
print(len(train_balanced_q), len(train_balanced_s), len(train_balanced_labels))

In [ ]:
from sklearn.utils import class_weight
EPOCHS = 5

#weights = class_weight.compute_class_weight('balanced', np.unique(train_labels[:1000]), train_labels[:1000])
hist = model.fit(x=[train_balanced_q[:,:,np.newaxis], train_balanced_s[:,:,np.newaxis]], y = train_balanced_labels, epochs = EPOCHS)

In [ ]:
y = model.predict(x=[train_balanced_q[:,:,np.newaxis], train_balanced_s[:,:,np.newaxis]])
print(np.where(y > 0.5))

In [ ]:
print(len(np.where(y > 0.8)[0]))